In [96]:
import pandas as pd
import numpy as np
import string,re,nltk

In [2]:
from nltk import WordNetLemmatizer
wn=WordNetLemmatizer()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [4]:
data=pd.read_csv('C:/Users/Akshat/Downloads/hate-speech-and-offensive-language-master/hate-speech-and-offensive-language-master/data/labeled_data.csv')

In [5]:
data.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [6]:
data.shape

(24783, 7)

In [7]:
data['class'].unique()

array([2, 1, 0], dtype=int64)

In [8]:
data.drop(data.columns[[0,1]],axis=1,inplace=True)

In [9]:
data.head(2)

,hate_speech,offensive_language,neither,class,tweet
0,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...


In [10]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].str.replace('\s+', ' ')
    df[text_field] = df[text_field].str.replace("\'", "")
    return df

In [11]:
data=standardize_text(data,'tweet')

In [12]:
data.head()

,hate_speech,offensive_language,neither,class,tweet
0,0,0,3,2,!!! rt as a woman you shouldnt complain about ...
1,0,3,0,1,!!!!! rt boy dats cold tyga dwn bad for cuffin...
2,0,3,0,1,!!!!!!! rt dawg!!!! rt you ever fuck a bitch a...
3,0,2,1,1,!!!!!!!!! rt she look like a tranny
4,0,6,0,1,!!!!!!!!!!!!! rt the shit you hear about me mi...


In [51]:
data.loc[data['class']==1]

,hate_speech,offensive_language,neither,class,tweet
1,0,3,0,1,!!!!! rt boy dats cold tyga dwn bad for cuffin...
2,0,3,0,1,!!!!!!! rt dawg!!!! rt you ever fuck a bitch a...
3,0,2,1,1,!!!!!!!!! rt she look like a tranny
4,0,6,0,1,!!!!!!!!!!!!! rt the shit you hear about me mi...
5,1,2,0,1,"!!!!!!!!!!!!!!!!!!"" the shit just blows me cla..."
6,0,3,0,1,"!!!!!!"" i can not just sit up and hate on anot..."
7,0,3,0,1,!!!! 8220 cause im tired of you big bitches co...
8,0,3,0,1,""" amp you might not get ya bitch back amp that..."
9,1,2,0,1,""" hobbies include fighting mariam"" bitch"
10,0,3,0,1,""" keeks is a bitch she curves everyone "" lol i..."


In [13]:
stopwords=nltk.corpus.stopwords.words('english')

In [14]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [15]:
X=data[data.columns.difference(['class'])]
Y=data['class']

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=3)

In [18]:
X_train.columns

Index(['hate_speech', 'neither', 'offensive_language', 'tweet'], dtype='object')

In [19]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text,ngram_range=(1,2))
tfidf_vect_fit = tfidf_vect.fit(X_train['tweet'])
tfidf_train = tfidf_vect_fit.transform(X_train['tweet'])
tfidf_test = tfidf_vect_fit.transform(X_test['tweet'])

In [20]:
# X_features = pd.concat([X_train['hate_speech'], X_train['neither'],X_train['offensive_language'], pd.DataFrame(tfidf_train.toarray())], axis=1)
# X_features.head()

In [21]:
from scipy.sparse import hstack
X_features = hstack((tfidf_train,np.array(X_train['hate_speech'])[:,None]))
X_1=hstack((X_features,np.array(X_train['neither'])[:,None]))
X_train_final=hstack((X_1,np.array(X_train['offensive_language'])[:,None]))

In [22]:
from scipy.sparse import hstack
X_features = hstack((tfidf_test,np.array(X_test['hate_speech'])[:,None]))
X_1=hstack((X_features,np.array(X_test['neither'])[:,None]))
X_test_final=hstack((X_1,np.array(X_test['offensive_language'])[:,None]))

In [23]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

In [65]:
nb_model=nb.fit(tfidf_train,y_train)

In [66]:
pred=nb_model.predict(tfidf_test)

In [26]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [37]:
print(accuracy_score(pred,y_test))

0.794028646359


In [38]:
print(classification_report(pred,y_test))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       1.00      0.79      0.88      4812
          2       0.16      0.94      0.28       145

avg / total       0.97      0.79      0.86      4957



C:\Users\Akshat\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [31]:
data['tweet'][0]

'!!! rt as a woman you shouldnt complain about cleaning up your house amp as a man you should always take the trash out '

In [69]:
sentences = [
    "You are a piece of shit, and I will step on you.",
    "THIS SUX!!!",
    "This kinda sux...",
    "You're good, man",
    'that bitch is crazyyyyy.. i dumped her','my ex was a jerk.. i kicked his ass','deport all muslims..let them go to hell!!',
    'you are amazing.. keep up','all politicians are crap and should be fireddddd','walking dead gonna be so trash this season'
            ]
#hash_vect_sample = TfidfVectorizer(analyzer=clean_text)
test = tfidf_vect_fit.transform(sentences)

In [67]:
test_pred=nb_model.predict(test)

In [68]:
for review, category in zip(sentences, test_pred):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => 1
'THIS SUX!!!' => 1
'This kinda sux...' => 1
"You're good, man" => 1
'HAHAHA YOU ARE THE BEST!!!!! VERY FUNNY!!!' => 1
'you are amazing.. keep up' => 1
'all politicians are crap and should be fireddddd' => 1
'walking dead gonna be so trash this season' => 1


In [52]:
from sklearn.linear_model import LogisticRegression

In [53]:
lr=LogisticRegression(C=0.1, class_weight='balanced', solver='newton-cg', random_state=40,
                               multi_class='multinomial')

In [61]:
lr_model=lr.fit(tfidf_train,y_train)

In [62]:
lr_pred=model.predict(tfidf_test)

In [57]:
print(accuracy_score(lr_pred,y_test))

0.831753076458


In [73]:
lr_test_pred=lr_model.predict(test)

In [71]:
for review, category in zip(sentences, lr_test_pred):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => 1
'THIS SUX!!!' => 2
'This kinda sux...' => 2
"You're good, man" => 2
'that bitch is crazyyyyy.. i dumped her' => 1
'my ex was a jerk.. i kicked his ass' => 1
'deport all muslims..let them go to hell!!' => 2
'you are amazing.. keep up' => 2
'all politicians are crap and should be fireddddd' => 2
'walking dead gonna be so trash this season' => 2


In [75]:
pd=pd.DataFrame(lr_test_pred)

In [1]:
param=[0.01,0.001,1,10,50,30,100,0.1]
accu=[]
for i in param:
    lr=LogisticRegression(C=i, solver='newton-cg', random_state=40)
    model=lr.fit(tfidf_train,y_train)
    predd=model.predict(tfidf_test)
    accu.append(accuracy_score(predd,y_test))

NameError: name 'LogisticRegression' is not defined

In [122]:
x=pd.DataFrame(accu,index=param)

In [127]:
x

,0
0.010,0.835788
0.001,0.832157
1.000,0.848094
10.000,0.861812
50.000,0.858180
30.000,0.859391
100.000,0.855961
0.100,0.831753


In [110]:
lr=LogisticRegression(C=0.1, class_weight='balanced', solver='newton-cg', random_state=40,
                               multi_class='multinomial')
model=lr.fit(tfidf_train,y_train)
lr_pred=model.predict(tfidf_test)

In [76]:
pd.loc[pd[0]==0, 0] = 'neutral'
pd.loc[pd[0]==1, 0] = 'offensive language'
pd.loc[pd[0]==2, 0] = 'hate speech'

In [77]:
for review, category in zip(sentences, pd[0]):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => offensive language
'THIS SUX!!!' => hate speech
'This kinda sux...' => hate speech
"You're good, man" => hate speech
'that bitch is crazyyyyy.. i dumped her' => offensive language
'my ex was a jerk.. i kicked his ass' => offensive language
'deport all muslims..let them go to hell!!' => hate speech
'you are amazing.. keep up' => hate speech
'all politicians are crap and should be fireddddd' => hate speech
'walking dead gonna be so trash this season' => hate speech


In [105]:
sentences = [
    "You are a piece of shit, and I will step on you.",
    "THIS SUX!!!",
    "This kinda sux...",
    "You're good, man",
    'that bitch is crazyyyyy.. i dumped her','my ex was a jerk.. i kicked his ass','deport all muslims..let them go to hell!!',
    'you are pathetic and disgusting!!','all politicians are crap and should be fireddddd','that movie was pathetic and sucked big time'
            ]
#hash_vect_sample = TfidfVectorizer(analyzer=clean_text)
test = tfidf_vect_fit.transform(sentences)

In [113]:
nb_pred=nb_model.predict(test)
lr_pre=model.predict(test)

In [114]:
nb_pred=pd.DataFrame(nb_pred)
lr_pre=pd.DataFrame(lr_pre)
nb_pred.loc[nb_pred[0]==0, 0] = 'neutral'
nb_pred.loc[nb_pred[0]==1, 0] = 'offensive language'
nb_pred.loc[nb_pred[0]==2, 0] = 'hate speech'
lr_pre.loc[lr_pre[0]==0, 0] = 'neutral'
lr_pre.loc[lr_pre[0]==1, 0] = 'offensive language'
lr_pre.loc[lr_pre[0]==2, 0] = 'hate speech'

In [108]:
for review, category in zip(sentences, nb_pred[0]):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => offensive language
'THIS SUX!!!' => offensive language
'This kinda sux...' => offensive language
"You're good, man" => offensive language
'that bitch is crazyyyyy.. i dumped her' => offensive language
'my ex was a jerk.. i kicked his ass' => offensive language
'deport all muslims..let them go to hell!!' => offensive language
'you are pathetic and disgusting!!' => offensive language
'all politicians are crap and should be fireddddd' => offensive language
'that movie was pathetic and sucked big time' => offensive language


In [109]:
for review, category in zip(sentences, lr_pre[0]):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => offensive language
'THIS SUX!!!' => hate speech
'This kinda sux...' => hate speech
"You're good, man" => hate speech
'that bitch is crazyyyyy.. i dumped her' => offensive language
'my ex was a jerk.. i kicked his ass' => offensive language
'deport all muslims..let them go to hell!!' => offensive language
'you are pathetic and disgusting!!' => hate speech
'all politicians are crap and should be fireddddd' => offensive language
'that movie was pathetic and sucked big time' => hate speech


In [115]:
for review, category in zip(sentences, lr_pre[0]):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => offensive language
'THIS SUX!!!' => hate speech
'This kinda sux...' => hate speech
"You're good, man" => hate speech
'that bitch is crazyyyyy.. i dumped her' => offensive language
'my ex was a jerk.. i kicked his ass' => offensive language
'deport all muslims..let them go to hell!!' => hate speech
'you are pathetic and disgusting!!' => hate speech
'all politicians are crap and should be fireddddd' => hate speech
'that movie was pathetic and sucked big time' => hate speech


In [116]:
from sklearn.ensemble import RandomForestClassifier

In [117]:
rf=RandomForestClassifier()
rf_model=rf.fit(tfidf_train,y_train)
rf_pred=rf_model.predict(tfidf_test)
print(accuracy_score(rf_pred,y_test))

0.869679241477


In [118]:
rf_pre=rf_model.predict(test)
rf_pre=pd.DataFrame(rf_pre)
rf_pre.loc[rf_pre[0]==0, 0] = 'neutral'
rf_pre.loc[rf_pre[0]==1, 0] = 'offensive language'
rf_pre.loc[rf_pre[0]==2, 0] = 'hate speech'

In [119]:
for review, category in zip(sentences, rf_pre[0]):
    print('%r => %s' % (review, category))

'You are a piece of shit, and I will step on you.' => offensive language
'THIS SUX!!!' => offensive language
'This kinda sux...' => offensive language
"You're good, man" => offensive language
'that bitch is crazyyyyy.. i dumped her' => offensive language
'my ex was a jerk.. i kicked his ass' => offensive language
'deport all muslims..let them go to hell!!' => offensive language
'you are pathetic and disgusting!!' => offensive language
'all politicians are crap and should be fireddddd' => offensive language
'that movie was pathetic and sucked big time' => offensive language
